In [ ]:
import carla
import pygame
import time
import random
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from carla import VehicleLightState

# Human-readable names for the 15 steering classes (strong left → no turn → strong right)
CLASS_NAMES = [
    "Left Hardest", "Left Harder", "Left Hard", "Left Medium", "Left Light", "Left Slight", "Left Minimal",
    "No Turning",
    "Right Minimal", "Right Slight", "Right Light", "Right Medium", "Right Hard", "Right Harder", "Right Hardest"
]

# ResNet18 backbone with a tiny head; turn signal goes through an embedding and is concatenated
class SteeringClassifier(nn.Module):
    def __init__(self, num_classes=15):
        super(SteeringClassifier, self).__init__()
        resnet = models.resnet18(pretrained=True)
        resnet.fc = nn.Identity()  # use the 512-d feature vector directly
        self.cnn = resnet
        self.turn_embed = nn.Embedding(3, 16)  # turn signals -1/0/1 are mapped to 0/1/2
        self.fc = nn.Sequential(
            nn.Linear(512 + 16, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, image, turn_signal):
        x_img = self.cnn(image)
        x_signal = self.turn_embed(turn_signal)
        x = torch.cat((x_img, x_signal), dim=1)
        return self.fc(x)

# Load the trained classifier and switch to eval mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SteeringClassifier(num_classes=15)
model.load_state_dict(torch.load("../Models/tertiary_model_final.pth", map_location=device))
model.eval().to(device)

# Image preprocessing to 224x224 + ImageNet normalization
image_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Pygame window for the live camera view + HUD text
pygame.init()
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("CARLA Model-Controlled Driving")

# Connect to the simulator and grab useful handles
client = carla.Client("localhost", 2000)
client.set_timeout(25.0)
client.load_world("Town05")
world = client.get_world()
blueprint_library = world.get_blueprint_library()

# Spawn a Tesla Model 3 at a random spawn point and keep manual control
vehicle_bp = blueprint_library.filter("vehicle.*model3*")[0]
spawn_point = random.choice(world.get_map().get_spawn_points())
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
vehicle.set_autopilot(False)

# We’ll keep the spectator following behind the car
spectator = world.get_spectator()

# Add a front RGB camera to the vehicle
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '448')
camera_bp.set_attribute('image_size_y', '252')
camera_bp.set_attribute('fov', '145')
camera_bp.set_attribute('sensor_tick', '0.1')
cam_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, cam_transform, attach_to=vehicle)

# Keep the latest frame as both a NumPy array (for the model) and a Pygame surface (for display)
camera_image = None
camera_np = None
pred_label = "Loading..."

def process_image(image):
    global camera_image, camera_np
    # CARLA gives BGRA bytes; convert to RGB NumPy and a Pygame surface
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    array = array[:, :, :3][:, :, ::-1]  # BGRA → RGB
    camera_np = array.copy()
    camera_image_raw = pygame.surfarray.make_surface(array.swapaxes(0, 1))
    camera_image = pygame.transform.scale(camera_image_raw, (800, 600))

# Start streaming camera frames
camera.listen(process_image)

# Small timing helper for the loop
clock = pygame.time.Clock()

print("Model-controlled driving started.")
print("Q/E to turn on left/right signal, R to cancel.")
print("ESC or close window to exit.")

# Map class IDs to steering values (range ~[-1, 1])
steering_map = {
    0: -1.0,   1: -0.8,   2: -0.6,   3: -0.4,   4: -0.25,  5: -0.15,  6: -0.05,
    7:  0.0,
    8:  0.05,  9:  0.15, 10:  0.25, 11:  0.4,  12:  0.6,  13:  0.8,  14:  1.0
}

# Main control loop: handle input, run the model, and apply vehicle commands
try:
    while True:
        clock.tick(30)
        pygame.event.pump()
        keys = pygame.key.get_pressed()

        # Turn signals: Q for left, E for right, R to clear; otherwise keep whatever is active
        if keys[pygame.K_q]:
            vehicle.set_light_state(VehicleLightState.LeftBlinker)
            turn_signal = -1
        elif keys[pygame.K_e]:
            vehicle.set_light_state(VehicleLightState.RightBlinker)
            turn_signal = 1
        elif keys[pygame.K_r]:
            vehicle.set_light_state(VehicleLightState.NONE)
            turn_signal = 0
        else:
            light_state = vehicle.get_light_state()
            if light_state == VehicleLightState.LeftBlinker:
                turn_signal = -1
            elif light_state == VehicleLightState.RightBlinker:
                turn_signal = 1
            else:
                turn_signal = 0

        # Ask the model for a steering class and convert it to a steering value
        control = carla.VehicleControl()
        control.throttle = 0.3

        if camera_np is not None:
            with torch.no_grad():
                img = image_transform(camera_np).unsqueeze(0).to(device)
                signal = torch.tensor([[turn_signal + 1]], dtype=torch.long).to(device)
                output = model(img, signal.squeeze(1))
                pred_class = output.argmax(dim=1).item()
                pred_label = CLASS_NAMES[pred_class]
                control.steer = steering_map[pred_class]

        vehicle.apply_control(control)

        # Keep the spectator camera trailing the car a few meters back and a bit above
        car_transform = vehicle.get_transform()
        forward_vector = car_transform.get_forward_vector()
        cam_location = car_transform.location - forward_vector * 8 + carla.Location(z=3)
        cam_rotation = carla.Rotation(pitch=-10, yaw=car_transform.rotation.yaw)
        spectator.set_transform(carla.Transform(cam_location, cam_rotation))

        # Draw the live camera view and the predicted class
        if camera_image:
            screen.blit(camera_image, (0, 0))
            font = pygame.font.SysFont(None, 30)
            label_surface = font.render(f"Predicted: {pred_label}", True, (255, 255, 0))
            screen.blit(label_surface, (20, 20))
            pygame.display.flip()

        # Close the window or hit ESC to exit cleanly
        for event in pygame.event.get():
            if event.type == pygame.QUIT or keys[pygame.K_ESCAPE]:
                raise KeyboardInterrupt

except KeyboardInterrupt:
    print("Exiting and cleaning up...")

finally:
    camera.stop()
    vehicle.destroy()
    camera.destroy()
    pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\aadav\anaconda3\envs\ACS\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\aadav\anaconda3\envs\ACS\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\aadav/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100.0%


FileNotFoundError: [Errno 2] No such file or directory: '../Models/steering_model_final.pth'

: 